# Моя база данных любимых фильмов
Вот как создать базу данных, включая дизайн БД и простую реализацию на python. Мы будем использовать SQLite3, который поставляется с Python, и, хотя он ограничен для небольших проектов БД, это непосредственный способ познакомиться с языком SQL, общим для всех программ для управления реляционными базами данных.

В качестве примера мы создадим базу данных для управления метаданными моих любимых фильмов, включая такие объекты, как актеры, режиссеры и награды. Цель состоит в том, чтобы сохранить всю эту информацию и иметь возможность задавать такие вопросы, как: какие из моих любимых фильмов сняты Квентином Тарантино? или какие фильмы с Джеффри Рашем имеют самый высокий рейтинг, которые я когда-либо смотрел?

## 1. Проектирование сущностно-реляционных диаграмм (ERD)
Прежде всего, нам нужно спроектировать архитектуру нашей БД. Самый простой способ — создать диаграмму отношений сущностей (Entity Relation Diagram, ERD). Это можно сделать онлайн с помощью ERDplus.com.

*(Хотя обучение проектированию ERD может показаться громоздким и ненужным в начале, оно становится все более и более необходимым по мере усложнения БД, и на самом деле ускоряет кодирование SQL, как мы увидим в разделе 3).*

<img src="https://github.com/faustodapuzzo/cimena_SQLite/blob/master/erd_cinema.png?raw=true" >

Диаграмма ERD представляет собой логическую структуру информации, подлежащей хранению. Хотя на первый взгляд это выглядит беспорядочно, его логику можно легко понять, например, «Актеры» связаны с «Фильмами», потому что они «снялись» в них, в то время как они не имеют прямой связи с «Режиссерами», кроме как через «Фильмы», которые были «Срежиссированы» ими.

Основная идея ERD заключается в том, чтобы разбить проект на:

1. Сущности (например, фильмы, актеры), которые являются самодостаточными логическими сущностями
2. Отношения (в бриллиантах, например, режиссер, кинопремия)
3. Атрибуты прикрепляются к сущностям или отношениям (в овале, например, титул, год, имя актера)
* Первичные ключи (подчеркнутые жирным шрифтом, например, actor_ID) являются наиболее распространенным примером уникального идентификатора и необходимыми элементами реляционной БД. Они позволяют обеспечить целостность данных и в конечном итоге создать связи между различными сущностями в БД
* Многозначные атрибуты (в двойном овале, например, у фильма может быть несколько продюсеров или рейтинг разных учреждений)

Соединительные линии между сущностями являются частью отношения, а символы на обоих концах называются кратностью и изображают, например, отношение «один-к-одному», «один-ко-многим» или «многие-ко-многим». Более подробную информацию о нотации ERD можно найти, например, здесь https://www.lucidchart.com/pages/ER-diagram-symbols-and-meaning.

## 2. Реляционная схема

В то время как ERD является логическим представлением проекта, более прямым способом представления того, как должна быть реализована БД, является реляционная схема. В схеме показаны «фактические» таблицы, в том числе для сущностей и связей, с дополнительной информацией о том, как соединить сущности с помощью ключей.

Обычно шаг от ERD к схеме является механическим, и ERDPlus.com предлагает инструмент для преобразования ERD в схему. И вот что у нас получается:

<img src="https://github.com/faustodapuzzo/cimena_SQLite/blob/master/schema_cinema.png?raw=true">

Здесь можно распознать ту же структуру, что и ERD, но с некоторыми отличиями:

1. Реляционные становятся таблицей с двумя Внешними Ключами (FK), если они не являются необязательными отношениями (например, не каждый актер или фильм имеет Награду)
2. Многозначные атрибуты становятся отдельными таблицами в соответствии с принципом атомарности БД (для получения дополнительной информации о принципах проектирования БД см. три правила нормализации)

Теперь мы готовы реализовать наш дизайн в реальную БД с помощью SQLite3.

## 3. Создайте базу данных SQLite3

Теперь мы можем использовать sqlite3 для создания файла db и открытия соединения:

In [1]:
import sqlite3

# this will create the .db file if it doesn't exist, and connect to it
conn = sqlite3.connect('cinema.db', timeout = 10)
# we define our cursor
c = conn.cursor()

Файл с именем cinema.db должен появиться в вашем рабочем каталоге. Этот файл будет содержать всю базу данных в двоичном формате. Чтобы открыть и изучить .db файл, скачайте "DB Browser for SQLite", который можно найти по адресу: https://github.com/sqlitebrowser/sqlitebrowser/releases

In [2]:
# Remember to close the cursor and connection when you are done with:
#c.close()
#conn.close()

Теперь пришло время заполнить БД таблицами и столбцами. Это требует знания языка SQL и будет объяснено по ходу дела.

Вот как мы создаем первую таблицу, синтаксические слова написаны заглавными буквами, определения таблиц и столбцов в нижнем регистре:

In [3]:
c.execute('CREATE TABLE IF NOT EXISTS Movies     \
            (movie_ID INTEGER PRIMARY KEY,       \
            title_en VARCHAR,                    \
            title_IT VARCHAR,                    \
            movie_year INT, \
            genre VARCHAR,                      \
            watched INT                          \
            );')

Базовый синтаксис для создания таблиц выглядит следующим образом:

**CREATE TABLE table_name (attribute1, attribute2,...)**

Обратите внимание, что за каждым атрибутом следует DataType (например, INT, VARCHAR), а в конце также указывается PRIMARY KEY. Аналогично можно создать и другие основные таблицы:

In [4]:
c.execute('\
    CREATE TABLE Directors            \
    (director_ID INTEGER PRIMARY KEY, \
    director_Fname VARCHAR NOT NULL,  \
    director_Lname VARCHAR NOT NULL,  \
    direc_birthYear INT               \
    );')
c.execute('\
    CREATE TABLE Actors               \
    (actor_ID INTEGER PRIMARY KEY,    \
    actor_Fname VARCHAR NOT NULL,     \
    actor_Lname VARCHAR NOT NULL,     \
    actor_birthYear INT               \
    );')

Остальные таблицы, относящиеся к отношениям сущностей и таблицам Awards, реализованы с той лишь разницей, что они включают в себя внешние ключи, которые определены с помощью интуитивно понятного синтаксиса команд (см. команды ниже).

In [5]:
c.execute("                                               \
    CREATE TABLE Awards                                   \
    (award_ID INTEGER PRIMARY KEY,                        \
    Oscar_year INT,                                       \
    Oscar_category VARCHAR,                               \
    OtherAward1_year INT,                                 \
    Highlight VARCHAR,                                    \
    actor_ID INT,                                         \
    movie_ID INT,                                         \
    FOREIGN KEY (actor_ID) REFERENCES Actors(actor_ID),   \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)    \
    );")

c.execute("                                                       \
    CREATE TABLE directed_by                                      \
    (director_ID INT NOT NULL,                                    \
    movie_ID INT NOT NULL,                                        \
    PRIMARY KEY (director_ID, movie_ID),                          \
    FOREIGN KEY (director_ID) REFERENCES Directors(director_ID),  \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)            \
    );")

c.execute("                                               \
    CREATE TABLE starred_in                               \
    (actor_ID INT NOT NULL,                               \
    movie_ID INT NOT NULL,                                \
    PRIMARY KEY (actor_ID, movie_ID),                     \
    FOREIGN KEY (actor_ID) REFERENCES Actors(actor_ID),   \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)    \
    );")                                                        #

c.execute("                                               \
    CREATE TABLE Movies_producers                         \
    (producer_name VARCHAR NOT NULL,                      \
    movie_ID INT NOT NULL,                                \
    PRIMARY KEY (producer_name, movie_ID),                \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)    \
    );")

c.execute('                                               \
    CREATE TABLE Movies_ratings                           \
    (ratings_ID INT NOT NULL,                             \
    Rotten_Tomatoes INT NOT NULL,                         \
    movie_ID INT NOT NULL,                                \
    PRIMARY KEY (ratings_ID, movie_ID),                   \
    FOREIGN KEY (movie_ID) REFERENCES Movies(movie_ID)    \
    );')

### ... ВАЖНЫЙ СОВЕТ:

Простой способ написать весь этот SQL-код, который может быть скучно делать вручную, — это экспортировать SQL-код из ERDPlus.com после разработки реляционной схемы. Это очень полезный бесплатный инструмент, который экономит много времени и утомительного, тщательного кодирования, и это сильный аргумент в пользу разработки ERD с подходящим программным обеспечением в первую очередь.

## 4. Заполнение данных

Перед заполнением базы данных убедитесь, что вы все еще подключены к базе данных, для повторного подключения вы можете использовать следующие команды:

In [6]:
conn = sqlite3.connect('cinema.db', timeout = 10)
c = conn.cursor()

Перед вставкой данных в таблицу может потребоваться проверить доступные столбцы и их имена. Вы можете изучить информацию таблицы (например, «Фильмы») с помощью следующих команд. Обратите внимание, что каждый раз, когда вы пытаетесь получить данные из БД, вам нужно отправить запрос, а затем fetchall() выбранные данные.

In [7]:
c.execute('PRAGMA table_info(Movies);') #
data = c.fetchall()
for row in data:
    print(row)

(0, 'movie_ID', 'INTEGER', 0, None, 1)
(1, 'title_en', 'VARCHAR', 0, None, 0)
(2, 'title_IT', 'VARCHAR', 0, None, 0)
(3, 'movie_year', 'INT', 0, None, 0)
(4, 'genre', 'VARCHAR', 0, None, 0)
(5, 'watched', 'INT', 0, None, 0)


Теперь вы можете вставить свою первую «строку» в одну из таблиц с помощью следующей команды:

**INSERT INTO table_name (column1, column2,...)  
VALUES (value1, value2, ...)**

In [8]:
c.execute("INSERT INTO Movies (title_en, movie_year) VALUES ('the daughter', 2015);")
conn.commit()


Обратите внимание, что после каждого INSERT данные должны быть зафиксированы, чтобы вставка вступила в силу. Также обратите внимание, что если первичный ключ определен как INTEGER, то его можно опустить в запросе INSERT, и будет присвоено автоматическое инкрементное значение.

In [9]:
c.execute("INSERT INTO Movies (title_en, movie_year, watched, genre) \
          VALUES ('the circle', 2017, 1, 'sci-fi'),                  \
                 ('the beauty and the beast', 2017, 1, 'fantasy'),   \
                 ('interstellar', 2014, 1, 'sci-fi'),                \
                 ('the martian', 2015, 1, 'sci-fi')                  \
          ;")
conn.commit()   # insert multiple movies with a single query

Давайте также вставим несколько акторов, а затем посмотрим, как мы можем связать таблицы с помощью внешних ключей:

In [10]:
c.execute("INSERT INTO Actors (actor_Fname, actor_Lname, actor_birthYear)  \
          VALUES ('Emma', 'Watson', 1990),                                 \
                 ('Matt', 'Damon', 1970)                                   \
          ;")
conn.commit()

Теперь мы готовы вставить данные в таблицы отношений, и, поскольку эти таблицы содержат внешние ключи, нам нужно убедиться, что мы правильно их извлекли, выбрав эти ключи из соответствующих таблиц. В этом случае мы будем использовать условие WHERE для их идентификации.

In [11]:
c.execute("INSERT INTO starred_in (actor_ID, movie_ID)   \
          SELECT actor_ID, movie_ID                      \
          FROM Actors, Movies                            \
          WHERE actor_Fname = 'Matt'                     \
          AND title_en = 'the martian';     \
          ")
conn.commit()

Заметьте, что было бы гораздо проще вставить в эту таблицу (и во все таблицы реляций), если бы PRIMARY KEY был известным нам значением, например, имя актера или название фильма. Тем не менее, мы должны знать его правильно и убедиться, что название фильма никогда не является двусмысленным (иначе не может работать как PK, потому что это не уникальный идентификатор). В этой ситуации часто разумнее использовать СУРРОГАТНЫЕ КЛЮЧИ, т.е. инкрементные целые числа, которые действуют как КЛЮЧ, хотя и делают запросы более вложенными для записи.

## 5. Запрос данных

Теперь мы готовы приступить к запросу данных и пожинать плоды нашей работы. Даже если БД содержит всего несколько записей, легко представить мощь этого подхода, когда ее размер увеличивается на много порядков.

Основная команда, чтобы научиться извлекать данные:

** SELECT column1, column2, ... FROM table_name**

In [12]:
c.execute("SELECT actor_Fname, actor_Lname FROM Actors;")
data = c.fetchall()
for row in data: print(row)

('Emma', 'Watson')
('Matt', 'Damon')


Обратите внимание, что опять же данные должны быть извлечены для печати.

Синтаксис команды SELECT очень богат, и вот команда::

**SELECT column  
FROM table_name  
WHERE condiiton  
GROUP BY column  
ORDER BY column  
LIMIT number**

Каждый столбец и таблица могут быть вызваны с помощью ALIAS (SELECT column1 AS c1) для упрощения запроса. WHERE поддерживает логическую логику, такую как AND, OR, IS, ISN, NOT, like regex..., а также GROUP BY и ORDER BY могут иметь несколько уровней, и, наконец, LIMIT поддерживает как смещение, так и диапазон.

Итак, как вы видите, синтаксис богат и позволяет проводить сложный анализ по мере введения других операторов, таких как SUM(), COUNT(), MAX(), AVERAGE(), etcc.

Вот пример более сложного запроса, где мы ищем актеров старше меня, чье имя начинается с буквы "m" (SQL не чувствителен к регистру!!).

In [13]:
c.execute("SELECT *                      \
          FROM Actors                    \
          WHERE actor_birthYear < 1986   \
          AND actor_Fname LIKE 'm%';     \
          " )
data = c.fetchall()
for row in data: print(row)

(2, 'Matt', 'Damon', 1970)


## 5. Запрос к нескольким таблицам с помощью "JOIN"

JOIN" является очень важной командой для более сложных запросов. Это позволяет по-настоящему использовать потенциал реляционных баз данных, выискивая определенные подмножества данных, распределенных по нескольким таблицам, сохраняя при этом вычислительную эффективность.

Существуют различные виды операций JOINING, которые соответствуют различным эквивалентам теории множеств:



**INNER JOIN = set intersection**  
**OUTER JOIN = set union**  
**RIGHT/LEFT JOIN = union with intersection**

Более подробную информацию можно найти на https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins.

Здесь мы продемонстрируем использование INNER JOIN только в двух случаях:

In [14]:
print('ID, Fname, Lname, Year, s_ID, m_ID')
c.execute("SELECT *                               \
          FROM Actors a                           \
          INNER JOIN starred_in s                 \
          ON a.actor_ID=s.actor_ID                \
          ;" )
data = c.fetchall()
for row in data: print(row)

ID, Fname, Lname, Year, s_ID, m_ID
(2, 'Matt', 'Damon', 1970, 2, 5)


Обратите внимание, что quary сообщает о первых 4 столбцах из таблицы Actors и последних 2 столбцах из таблицы starred_in. Поэтому здесь мы соединили таблицы "Actors" и "starred_in", связав их с оператором "ON", который указывает, какие столбцы должны быть сравнены для связывания.

В результате мы нашли пересечение двух таблиц, по сути, мы не получили имени какого-либо другого актера, потому что в текущей БД они не привязаны ни к одному фильму.

#### Crossing several relationship levels

If we want to select actors that star in a certain movie (and we don't know the movie_ID off the top of our head) we have to reach across one more level to make this connection. Please notice how Actors and Movies are not directly linked in the ERD:

<img src="erd_cinema_highlight.png" style="width: 500px;"/>

This operation can be done by nesting as many JOIN statements as the links that have to be crossed in the ERD. In this example we will start from the Actors and select only those that intersect with a set made of the intersection between the "starred_in" table and the "Movies" set where our condition on the title is set.  

So, let's SELECT the Actors that star into the movie "the beauty and the beast":

In [15]:
c.execute("SELECT * FROM Movies m INNER JOIN (         \
          SELECT movie_ID FROM starred_in s            \
          INNER JOIN Actors a ON a.actor_ID=s.actor_ID \
          WHERE a.actor_Lname = 'Watson'\
          ) sm ON m.movie_ID=sm.movie_ID;" )
data = c.fetchall()
for row in data: print(row)

It will look cryptic for the first 10 minutes, and it's OK. To help you crack it, notice how the first INNER JOIN intersects the Actors with a set created by the query itself and contained between "()". This intermediate set is created with another SELECT statement that contains a nested INNER JOIN between the "starred_in" and the "Movies" table.

#### How to add an extra column

In [16]:
c.execute("ALTER TABLE Movies ADD COLUMN comment VARCHAR;" )
# In theory I would use a BOOLEAN type but SQLite doesn't have it
conn.commit()

In [17]:
c.execute('PRAGMA table_info(Movies);')
data=c.fetchall()
data

[(0, 'movie_ID', 'INTEGER', 0, None, 1),
 (1, 'title_en', 'VARCHAR', 0, None, 0),
 (2, 'title_IT', 'VARCHAR', 0, None, 0),
 (3, 'movie_year', 'INT', 0, None, 0),
 (4, 'genre', 'VARCHAR', 0, None, 0),
 (5, 'watched', 'INT', 0, None, 0),
 (6, 'comment', 'VARCHAR', 0, None, 0)]

**More population quaryies**

In [18]:
#c.execute("INSERT INTO Movies (title_en, movie_year, watched, genre) \
 #         VALUES ('jurassic park',2017, 1, 'sci-fi');")

In [19]:
# Multiple MOVIES
c.execute("INSERT INTO Movies (title_en, movie_year, watched, genre) \
          VALUES ('jurassic park',2017, 1, 'sci-fi');")
conn.commit()   # insert multiple movies with a single query

In [ ]:
# Multiple ACTORS
c.execute("INSERT INTO Actors (actor_Fname, actor_Lname, actor_birthYear) \
          VALUES ('Geoffrey', 'Rush', 1951),                             \
                 ('Sam', 'Neill', 1947),                           \
                 ('Miranda', 'Otto', 1967),                \
                 ('Uma', 'Thurmann', 1970),                                \
                 ('John', 'Travolta', 1954),         \
                 ('Samuel', 'Jackson', 1948),           \
                 ('Christof', 'Walts', 1956),           \
                 ('Bruce', 'Willis', 1955),             \
                 ('Christopher', 'Walken', 1943),       \
                 ('Quentin', 'Tarantino', 1963),        \
                 ('Leonardo', 'Di Caprio', 1974),    \
                 ('Jack', 'Nicholson', 1937),         \
                 ('Morgan', 'Freeman', 1937),           \
                 ('Tom', 'Hanks', 1956),          \
                 ('Owen', 'Wilson', 1968),            \
                 ('Cameron', 'Diaz', 1972),          \
                 ('Jim', 'Carrey', 1962),          \
                 ('Jim', 'Broadbent', 1949),    \
                 ('Liam', 'Neeson', 1952),      \
                 ('Will', 'Smith', 1968),            \
                 ('Robert', 'Downey', 1965),          \
                 ('Kirsten', 'Dunst', 1982),         \
                 ('Chris', 'Evans', 1981),          \
                 ('Mark', 'Ruffalo', 1967),        \
                 ('Scarlett', 'Johannson', 1984),   \
                 ('Gwynett', 'Paltrow', 1972),     \
                 ('Liv', 'Tyler', 1977)        \
          ;")
conn.commit()   # insert multiple movies with a single query

In [20]:
# MOVIE ACTOR RELATIONSHIP
c.execute("INSERT INTO starred_in (actor_ID, movie_ID)   \
          SELECT actor_ID, movie_ID                      \
          FROM Actors, Movies                            \
          WHERE actor_Lname = 'Corbett'                     \
          AND title_IT LIKE 'il mio grosso%';     \
          ")
conn.commit()

In [21]:
c.execute("INSERT INTO Directors (director_Fname, director_Lname)  \
          VALUES ('Quentin', 'Tarantino'),                         \
          ('Martin', 'Scorsese'),                                  \
          ('Steven', 'Spielberg'),                                 \
          ('Christopher', 'Nolan'),                                \
          ('Stanley', 'Kybrick'),                                  \
          ('Alfred', 'Hitchcock'),                                 \
          ('Woody', 'Allen')                                       \
          ")
conn.commit()

In [22]:
# MOVIE-director RELATIONSHIP
c.execute("INSERT INTO directed_by (director_ID, movie_ID)   \
          SELECT director_ID, movie_ID                      \
          FROM Directors, Movies                            \
          WHERE director_Lname = 'Spielberg'                     \
          AND title_en LIKE 'save%';     \
          ")
conn.commit()

In [23]:
c.execute("INSERT OR IGNORE INTO Movies (title_en, movie_year, watched, genre) \
          VALUES ('jurassic park', 1993, 1, 'sci-fi');")
conn.commit()

Here is how to delete entries:

In [24]:
c.execute("DELETE FROM Movies WHERE title_en LIKE 'jurassic%';")
conn.commit()

Here is how to check how many rows have been affected by the last changes, and check if an INSERT or DELECT, ecc has been successful.

In [25]:
c.execute("SELECT changes();")
result = c.fetchall()
print(result)

[(2,)]


## 6. Let's automate the process a bit

Now this is great and fun, but every query is a lot of writing, and we might want to repeat the same or a similar quary for many different values, both in data entry and in data analysis. So what we have to do is to create customized functions that will "edit" the query for us. And eventually programmatically perform data entry from verious structured or unstructured lists.

First of all we need to start by creating low level functions that perform basic operations, like inserting in a single value in a single table from structured variables, and from there create higher level functions that perform more flexible operations.

In [26]:
def singleTable_entry(table, column, data):
    """This function formats the query and commits an entry to DB

    Parameters:
        table = 'table_name' as a string
        column = ['colum1', 'column2',..] is a list of strings
        data = [123, 'name', 456, ...] is a list of strings and integers

    Returns:
        "str" with SQLite command to insert DATA into COLUMN
    """

    for i in range(len(data)):

        # check for datatype and format accordingly
        if isinstance(data[i], str):
            data[i] = str("'" + data[i] + "'")

        elif isinstance(data[i], int):
            data[i] = str(data[i])

    # join list elements into one string
    column = ', '.join(column)
    data = ', '.join(data)

    #create command string
    cmd = 'INSERT INTO %s (%s) VALUES(%s);' %(table, column, data)

    return cmd

with this function we can get the query already fromatted starting from structured data like so:

In [27]:
# some structured data
table = 'my_table'
column = ['my_column1', 'my_column2', 'my_column3', 'my_column3']
value = ['bablabal', 123456, 'blabla2', 58]

#let's use the function on this data
singleTable_entry(table, column, value)

"INSERT INTO my_table (my_column1, my_column2, my_column3, my_column3) VALUES('bablabal', 123456, 'blabla2', 58);"

Notice that the function has been written to recognize two datatypes (str, int) and to format the query to respect that.

In [28]:
import sqlite3

conn = sqlite3.connect('cinema.db', timeout = 10)
c = conn.cursor()

In [29]:
c.close()
conn.close()